Tag-and-Probe is a data-driven technique used to calculate lepton reconstruction, identification, and trigger efficiencies. This technique uses narrow dilepton resonances, such as Z (for muons with relatively high pT) or J/ψ (for muons with lower pT). Almost-unbiased estimates of the efficiencies can be obtained at the different stages of muon trigger and offline reconstruction. Events are selected with strict requirements on one muon (the tag), and with a more relaxed selection on the other muon (the probe), such that the probe muon can be used to measure the efficiency in question without large biases. The fraction of probe muons that pass the selection under study gives an estimate of its efficiency. The invariant mass of the tag-probe pair is used to select Z→μμ or J/ψ→μμ events.
The Tag-and-Probe technique is generally used to measure and compare efficiencies in data and in MC simulation, and thus to compute a correction scale factor that can be applied to MC events to match the efficiency observed in data. These scale factors are typically determined as functions of pT and η. If necessary, their dependence on other kinematic variables can be investigated too — e.g. vs the number of vertices, in case of strong pileup dependence. In some cases, customized scale factors are necessary for some analyses, depending on their specific trigger and offline thresholds.

Despite the tight selection on the tag muon and the invariant mass constraints, the selected Z→μμ or J/ψ→μμ sample generally contains background events, which appear as a nonresonant continuum underneath the resonance peak. Therefore the background must be subtracted, to ensure that the efficiency is measured with signal muons only. This can be achieved by fitting the invariant mass spectrum to signal + background shapes (e.g. analytical functions or MC templates). Finding proper functions or templates for signal and background is often the most challenging part of the process.

The total lepton efficiency is generally factorized in multiple steps as follows:
total lepton efficiency = (tracking) × (reconstruction/tracking) × (ID/reconstruction) × (isolation/ID) × (trigger/isolation)
In each efficiency step, the denominator determines the selection of the probe. In the last steps (in part. isolation and trigger), the probe selection is tighter and, therefore, the background level is quite low and the background subtraction is generally easier or not even needed, in some cases. In this exercise, you will measure efficiencies using simulated Z→μμ events:

DYJetsToMuMu_M-50_TuneCP5_14TeV-madgraphMLM-pythia8.root

Since you are using a pure Z sample, you won't need background subtraction nor fitting. The efficiency will simply be computed by counting probes before and after the selection under study.

You will start by computing "true" efficiencies using the generator-level information. This will be your reference efficiencies. Next, you will implement a simple tag-and-probe algorithm to measure the efficiencies with a data-driven approach, and you will compare your results to the "true" efficiencies. Finally, you will try to use the same algorithm on a real single-muon data sample:

Run2016G_SingleMuon_NANOAOD.root<br>
This sample stored 1,295,544 events.


In [ ]:
import uproot
import ROOT
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak
import vector
vector.register_awkward()
from math import pi, sqrt, cosh, cos

In [ ]:
dR=0.1 #set delatr requirement

# Step 1: "MC-true" efficiency

First,  select the prompt final state gen level muons with pT > 20 GeV and |η| < 2.4, and fill histograms with the main kinematic variables. Include a distribution of the number of reconstructed vertices

Next, select reco muons from Muon collections and check that they are matched to one of the GenParticle muons used . Then you can start filling histograms (using the generator-level information from the associated GenParticle) at different selection stages:
* all reconstructed muons with a valid inner track (i.e. skip muons that are reconstructed exclusively as standalone muons)
* reconstructed muons passing some standard ID: e.g. loose, medium, tight
* tight muons that pass a requirement on the Δβ-corrected combined relative isolation (e.g. < 0.12)
* isolated muons that pass a single-muon trigger or a combination of single-muon triggers, e.g. HLT_IsoMu24, HLT_IsoTkMu24 This step requires matching the muon candidate to a trigger object.

Finally, create efficiency graphs by computing the ratio of the histograms above. You can use the TGraphAsymmErrors class, which gives you different options to compute the errors on the efficiency.

Compute the following factorized efficiencies:
* reconstructed muons / generated muons
* muon ID / reconstructed muons
* isolation / muon ID
* single-muon trigger / isolation

In [ ]:
#file=uproot.open('~/Downloads/DYJetsToMuMu_M-50_TuneCP5_14TeV-madgraphMLM-pythia8.root')['Events']
file=uproot.open('/cms/store/user/hats/Muon/DYJetsToMuMu_M-50_TuneCP5_14TeV-madgraphMLM-pythia8.root')['Events']

selelct gen particle

In [ ]:
genpart_mask=(file['GenPart_pt'].array()>20)& (abs(file['GenPart_eta'].array())<2.4)& (abs(file['GenPart_pdgId'].array())==13)& (file['GenPart_status'].array()==1)& ((file['GenPart_statusFlags'].array()%2)==1)

In [ ]:
gen_muon=ak.Array(ak.zip({"pt": file['GenPart_pt'].array()[genpart_mask], 
                            "eta": file['GenPart_eta'].array()[genpart_mask], 
                            "phi": file['GenPart_phi'].array()[genpart_mask],
                            "npv": file['PV_npvsGood'].array()}))
                             
                             

In [ ]:
muon=ak.Array(ak.zip({"muon_pt":file['Muon_pt'].array(), "muon_eta": file['Muon_eta'].array(), "muon_phi": file['Muon_phi'].array(),
                        "muon_isglobal": file['Muon_isGlobal'].array(), "muon_istracker": file['Muon_isTracker'].array(),
                        "muon_mass": file['Muon_mass'].array(),
                        "muon_charge": file['Muon_charge'].array(),
                        "muon_dxy":file['Muon_dxy'].array(),
                        "muon_dz":file['Muon_dz'].array(),
                        "muon_pfreliso04_all": file['Muon_pfRelIso04_all'].array(), 
                        "muon_tkreliso": file['Muon_tkRelIso'].array(),
                        "muon_looseid": file['Muon_looseId'].array(), 
                        "muon_mediumid": file['Muon_mediumId'].array(), 
                        "muon_tightid": file['Muon_tightId'].array(), 
                        "muon_genpartidx": file['Muon_genPartIdx'].array(),
                        "pv_npvsgood": file['PV_npvsGood'].array(),
                        "genpart_pt": file['GenPart_pt'].array()[file['Muon_genPartIdx'].array()],
                        "genpart_eta": file['GenPart_eta'].array()[file['Muon_genPartIdx'].array()],
                        "genpart_phi": file['GenPart_phi'].array()[file['Muon_genPartIdx'].array()],
                        "genpart_pdgid": file['GenPart_pdgId'].array()[file['Muon_genPartIdx'].array()],
                        "genpart_status": file['GenPart_status'].array()[file['Muon_genPartIdx'].array()],
                        "genpart_statusFlags": file['GenPart_statusFlags'].array()[file['Muon_genPartIdx'].array()],
                     }))


In [ ]:
trigger_muon=ak.Array(ak.zip({"pt":file['TrigObj_pt'].array()[abs(file['TrigObj_id'].array())==13],
                                "eta": file['TrigObj_eta'].array()[abs(file['TrigObj_id'].array())==13], 
                                "phi": file['TrigObj_phi'].array()[abs(file['TrigObj_id'].array())==13]}))

In [ ]:
def deltaR(genetas, eta, genphis, phi):
    return np.sqrt((genetas-eta)**2+((genphis-phi+pi)%(2*pi)-pi)**2)

In [ ]:
def deltar_matching(genpart_eta, genpart_phi, reco_eta, reco_phi):
    list=[]
    for genetas, genphis, etas, phis in zip(genpart_eta, genpart_phi, reco_eta, reco_phi):
        i=[]
        if len(etas)==0:
            list.append([])
            continue
        
        for eta, phi in zip(etas, phis):
            if len(genetas)==0:
                i.append(False)
            else:
                deltar=deltaR(genetas, eta, genphis, phi)
                if (ak.min(deltar)<dR):
                     i.append(True)
                else:
                     i.append(False)    
        
        list.append(i)
    return ak.Array(list)
    

In [ ]:
c={}
def TH1_fill_and_plot(th1, array, name): #fill and plot TH1
    
    for a in ak.flatten(array):
        th1.Fill(a)
        
    c[f'{name}']= ROOT.TCanvas(f'{name}',"",900,600)
    th1.Draw()
    c[f'{name}'].Draw()

In [ ]:
c1={}
def eff_fill_and_plot(th1, name): #plot the efficiency graph
     
    c1[f'{name}']= ROOT.TCanvas(f'{name}',"",900,600)
    th1.Draw()
    c1[f'{name}'].Draw()

Now lets set a few cut on the event

* Event cut: must has at least a good vertex
* Particle cut: 

* 1. correspond genpart must be good muon 
* 2. fired a trigger

In [ ]:
good_gen_mask=((muon.muon_isglobal==1)|(muon.muon_istracker==1))& (muon.muon_genpartidx>-1)& (muon.genpart_pt>20)& (abs(muon.genpart_eta)<2.4)& (abs(muon.genpart_pdgid)==13)& (muon.genpart_status==1)& ((muon.genpart_statusFlags%2)==1)

In [ ]:
event_mask_goodpv= file['PV_npvsGood'].array()>0 #at least a good vertex

In [ ]:
event_mask_goodpv_HLT_isomu24= (file['PV_npvsGood'].array()>0)& (file['HLT_IsoMu24'].array()==1) #fired the iso muon trigger

In [ ]:
trigger_match_mask=deltar_matching(trigger_muon.eta, trigger_muon.phi, muon.muon_eta, muon.muon_phi) #match to a trigger muon

In [ ]:
muon_mask={}
muon_mask['rec']= good_gen_mask
muon_mask['rec_loose']= good_gen_mask& (muon.muon_looseid==1) 
muon_mask['rec_medium']= good_gen_mask& (muon.muon_mediumid==1) 
muon_mask['rec_tight']= good_gen_mask& (muon.muon_tightid==1)
muon_mask['rec_tight_iso']= good_gen_mask& (muon.muon_tightid==1)& (muon.muon_pfreliso04_all< 0.12)
muon_mask['rec_tight_iso_hlt']= good_gen_mask& (muon.muon_tightid==1)& (muon.muon_pfreliso04_all< 0.12)& trigger_match_mask

In [ ]:
th1dic={} #dictionary of TH1D
efficiencydic={} #dictionary of TGraphAsymmErrors
for name in ['gen', 'rec', 'rec_loose', 'rec_medium', 'rec_tight', 'rec_tight_iso', 'rec_tight_iso_hlt']:
    if name== 'gen':           #Cannot plot efficiency for gen itself, so if 
        th1dic[f'h_{name}_pt']=ROOT.TH1D("pt"  , f'h_{name}_pt;muon p_T [GeV];Events',  20,  20.0, 120.0) #constrcut
        TH1_fill_and_plot(th1dic[f'h_{name}_pt'], gen_muon.pt, f'h_{name}_pt') #fill and plot
        th1dic[f'h_{name}_eta']=ROOT.TH1D("eta" , f'h_{name}_eta;muon #eta;Events',  24,  -2.4,   2.4)
        TH1_fill_and_plot(th1dic[f'h_{name}_eta'], gen_muon.eta, f'h_{name}_eta')
        th1dic[f'h_{name}_nvtx'] =ROOT.TH1D("nvtx", f'h_{name}_nvtx;N. vertices;Events',  90,   0.5,  90.5)
        TH1_fill_and_plot(th1dic[f'h_{name}_nvtx'], gen_muon.npv, f'h_{name}_nvtx')
        
        name2=name
        continue
        
    if name== 'rec_tight_iso_hlt':
        th1dic[f'h_{name}_pt']=ROOT.TH1D("pt"  , f'h_{name}_pt;muon p_T [GeV];Events',  20,  20.0, 120.0)
        TH1_fill_and_plot(th1dic[f'h_{name}_pt'], muon.genpart_pt[muon_mask[f'{name}']][event_mask_goodpv_HLT_isomu24], f'h_{name}_pt')
        th1dic[f'h_{name}_eta']=ROOT.TH1D("eta" , f'h_{name}_eta;muon #eta;Events',  24,  -2.4,   2.4)
        TH1_fill_and_plot(th1dic[f'h_{name}_eta'], muon.genpart_eta[muon_mask[f'{name}']][event_mask_goodpv_HLT_isomu24], f'h_{name}_eta')
        th1dic[f'h_{name}_nvtx'] =ROOT.TH1D("nvtx", f'h_{name}_nvtx;N. vertices;Events',  90,   0.5,  90.5)
        TH1_fill_and_plot(th1dic[f'h_{name}_nvtx'], muon.pv_npvsgood[muon_mask[f'{name}']][event_mask_goodpv_HLT_isomu24], f'h_{name}_nvtx')

        efficiencydic[f'gae_{name}_pt']   = ROOT.TGraphAsymmErrors(20) 
        efficiencydic[f'gae_{name}_pt'].SetName(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].GetXaxis().SetTitle(f'h_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_pt'].Divide(th1dic[f'h_{name}_pt']  , th1dic[f'h_{name2}_pt']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_pt'], f'gae_{name}_pt')
        
        efficiencydic[f'gae_{name}_eta']   = ROOT.TGraphAsymmErrors(24 ) 
        efficiencydic[f'gae_{name}_eta'].SetName(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_eta'].GetXaxis().SetTitle(f'h_{name}_eta')
        efficiencydic[f'gae_{name}_eta'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_eta'].Divide(th1dic[f'h_{name}_eta']  , th1dic[f'h_{name2}_eta']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_eta'], f'gae_{name}_eta')
        
        efficiencydic[f'gae_{name}_nvtx']   = ROOT.TGraphAsymmErrors(90 ) 
        efficiencydic[f'gae_{name}_nvtx'].SetName(f'gae_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].GetXaxis().SetTitle(f'h_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_nvtx'].Divide(th1dic[f'h_{name}_nvtx']  , th1dic[f'h_{name2}_nvtx']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_nvtx'], f'gae_{name}_nvtx')
        
        name2=name
    else:
        th1dic[f'h_{name}_pt']=ROOT.TH1D("pt"  , f'h_{name}_pt;muon p_T [GeV];Events',  20,  20.0, 120.0)
        TH1_fill_and_plot(th1dic[f'h_{name}_pt'], muon.genpart_pt[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_pt')
        th1dic[f'h_{name}_eta']=ROOT.TH1D("eta" , f'h_{name}_eta;muon #eta;Events',  24,  -2.4,   2.4)
        TH1_fill_and_plot(th1dic[f'h_{name}_eta'], muon.genpart_eta[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_eta')
        th1dic[f'h_{name}_nvtx'] =ROOT.TH1D("nvtx", f'h_{name}_nvtx;N. vertices;Events',  90,   0.5,  90.5)
        TH1_fill_and_plot(th1dic[f'h_{name}_nvtx'], muon.pv_npvsgood[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_nvtx')
        
        efficiencydic[f'gae_{name}_pt']   = ROOT.TGraphAsymmErrors(20 ) 
        efficiencydic[f'gae_{name}_pt'].SetName(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].GetXaxis().SetTitle(f'h_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_pt'].Divide(th1dic[f'h_{name}_pt']  , th1dic[f'h_{name2}_pt']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_pt'], f'gae_{name}_pt')
        
        efficiencydic[f'gae_{name}_eta']   = ROOT.TGraphAsymmErrors(24 ) 
        efficiencydic[f'gae_{name}_eta'].SetName(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_eta'].GetXaxis().SetTitle(f'h_{name}_eta')
        efficiencydic[f'gae_{name}_eta'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_eta'].Divide(th1dic[f'h_{name}_eta']  , th1dic[f'h_{name2}_eta']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_eta'], f'gae_{name}_eta')
        
        efficiencydic[f'gae_{name}_nvtx']   = ROOT.TGraphAsymmErrors(90 ) 
        efficiencydic[f'gae_{name}_nvtx'].SetName(f'gae_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].GetXaxis().SetTitle(f'h_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_nvtx'].Divide(th1dic[f'h_{name}_nvtx']  , th1dic[f'h_{name2}_nvtx']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_nvtx'], f'gae_{name}_nvtx')
        
        name2=name



# Step 2 "Tag-and-Probe" efficiency

Note that, in this case, you will not be accessing any generator-level information. This analyzer could be used on real data

First, select the tag with the following cuts:
* Tight muon ID
* pT > 25 GeV and |η| < 2.4
* Combined relative PF isolation < 0.12 (in a cone ΔR < 0.4)
* Match a trigger object passing a single-muon trigger (e.g. HLT_IsoMu24)

Once you have found a tag, you can search for a probe. To be consistent with the previous part of the exercise, the probe should be any muon having pT > 20 GeV and |η| < 2.4 with a valid inner track. Additionally, to ensure that the tag-probe pair is compatible with a Z→μμ decay, require that tag and probe have opposite electric charges, and that their invariant mass is within a narrow window around the nominal Z mass. Since you are not performing any background subtraction here, you should choose a very narrow window, to reduce the level of background (in real data). E.g. you can choose (91 ± 5) GeV.

Once you have found both a tag and a probe, you can proceed to fill the plots using the probe information, at different stages of probe selection. Make the same plots as in Part 1, starting from the reconstructed muons with a valid track, obviously you don't have a collection of "all the generated muons" in this case! Create the efficiency TGraphAsymmErrors as in Part 1, starting from the ID efficiencies.

Finally, run the code and compare the tag-and-probe efficiencies with the MC-truth efficiencies.

In [ ]:
tag_muon_mask=(muon.muon_pt>25)& (abs(muon.muon_eta)<2.4)& (muon.muon_tightid==1)& (muon.muon_dz<0.1)& (muon.muon_pfreliso04_all< 0.12)& trigger_match_mask

In [ ]:
tag_muon=ak.Array(ak.zip({"pt": muon.muon_pt[tag_muon_mask],
                            "eta": muon.muon_eta[tag_muon_mask],
                            "phi" : muon.muon_phi[tag_muon_mask],
                            "mass": muon.muon_mass[tag_muon_mask],
                            "charge": muon.muon_charge[tag_muon_mask]}))

In [ ]:
probe_muon_mask=[]
# make sure the z mass is within 91+-5 window
for tag4s, probe4s, in zip(tag_muon, muon):
    list=[]
    if len(probe4s)==0:
        probe_muon_mask.append([])
        continue
    for probe4 in probe4s:
        bestmass=0
        if len(tag4s)==0:
            list.append(False)
            continue
        for tag4 in tag4s:
            if tag4.charge*probe4.muon_charge>0:
                continue
            mass=sqrt(2*tag4.pt*probe4.muon_pt*(cosh(tag4.eta - probe4.muon_eta) - cos(tag4.phi - probe4.muon_phi)))
            if abs(mass-91.)>5.:
                continue
            else:
                bestmass= mass
                break
        if bestmass!=0:
            list.append(True)
        else:
            list.append(False)
    probe_muon_mask.append(list)
            
            

In [ ]:
probe_muon_masking=ak.Array(probe_muon_mask)& (muon.muon_pt>20)& (abs(muon.muon_eta)<2.4)

In [ ]:
probe_muon= ak.Array(ak.zip({"pt": muon.muon_pt[probe_muon_masking],
                               "eta": muon.muon_eta[probe_muon_masking],
                               "phi": muon.muon_phi[probe_muon_masking],
                               "npvs": muon.pv_npvsgood[probe_muon_masking]}))

In [ ]:
muon_mask={}
muon_mask['rec']= probe_muon_masking
muon_mask['rec_loose']= probe_muon_masking& (muon.muon_looseid==1) 
muon_mask['rec_medium']= probe_muon_masking& (muon.muon_mediumid==1) 
muon_mask['rec_tight']=  probe_muon_masking& (muon.muon_tightid==1)& (muon.muon_mediumid==1)
muon_mask['rec_tight_iso']= probe_muon_masking& (muon.muon_tightid==1)& (muon.muon_pfreliso04_all< 0.12)& (muon.muon_mediumid==1)
muon_mask['rec_tight_iso_hlt']= probe_muon_masking& (muon.muon_tightid==1)& (muon.muon_pfreliso04_all< 0.12)& trigger_match_mask& (muon.muon_mediumid==1)

In [ ]:
th1dic={}
efficiencydic={}
for name in ['rec', 'rec_loose', 'rec_medium', 'rec_tight', 'rec_tight_iso', 'rec_tight_iso_hlt']:
    if name== 'rec':
        th1dic[f'h_{name}_pt']=ROOT.TH1D("pt"  , f'h_{name}_pt;muon p_T [GeV];Events',  20,  20.0, 120.0)
        TH1_fill_and_plot(th1dic[f'h_{name}_pt'], muon.muon_pt[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_pt')
        th1dic[f'h_{name}_eta']=ROOT.TH1D("eta" , f'h_{name}_eta;muon #eta;Events',  24,  -2.4,   2.4)
        TH1_fill_and_plot(th1dic[f'h_{name}_eta'], muon.muon_eta[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_eta')
        th1dic[f'h_{name}_nvtx'] =ROOT.TH1D("nvtx", f'h_{name}_nvtx;N. vertices;Events',  90,   0.5,  90.5)
        TH1_fill_and_plot(th1dic[f'h_{name}_nvtx'], muon.pv_npvsgood[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_nvtx')
        
        name2=name
        continue
        
    if name== 'rec_tight_iso_hlt':
        th1dic[f'h_{name}_pt']=ROOT.TH1D("pt"  , f'h_{name}_pt;muon p_T [GeV];Events',  20,  20.0, 120.0)
        TH1_fill_and_plot(th1dic[f'h_{name}_pt'], muon.muon_pt[muon_mask[f'{name}']][event_mask_goodpv_HLT_isomu24], f'h_{name}_pt')
        th1dic[f'h_{name}_eta']=ROOT.TH1D("eta" , f'h_{name}_eta;muon #eta;Events',  24,  -2.4,   2.4)
        TH1_fill_and_plot(th1dic[f'h_{name}_eta'], muon.muon_eta[muon_mask[f'{name}']][event_mask_goodpv_HLT_isomu24], f'h_{name}_eta')
        th1dic[f'h_{name}_nvtx'] =ROOT.TH1D("nvtx", f'h_{name}_nvtx;N. vertices;Events',  90,   0.5,  90.5)
        TH1_fill_and_plot(th1dic[f'h_{name}_nvtx'], muon.pv_npvsgood[muon_mask[f'{name}']][event_mask_goodpv_HLT_isomu24], f'h_{name}_nvtx')

        nbinpt   = th1dic['h_rec_pt'].GetNbinsX() 
        nbineta  = th1dic['h_rec_eta'].GetNbinsX() 
        nbinnvtx = th1dic['h_rec_nvtx'].GetNbinsX()
        
        efficiencydic[f'gae_{name}_pt']   = ROOT.TGraphAsymmErrors(nbinpt  ) 
        efficiencydic[f'gae_{name}_pt'].SetName(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].SetTitle(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].GetXaxis().SetTitle(f'h_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_pt'].Divide(th1dic[f'h_{name}_pt']  , th1dic[f'h_{name2}_pt']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_pt'], f'gae_{name}_pt')
        
        efficiencydic[f'gae_{name}_eta']   = ROOT.TGraphAsymmErrors(nbineta ) 
        efficiencydic[f'gae_{name}_eta'].SetName(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_eta'].SetTitle(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_eta'].GetXaxis().SetTitle(f'h_{name}_eta')
        efficiencydic[f'gae_{name}_eta'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_eta'].Divide(th1dic[f'h_{name}_eta']  , th1dic[f'h_{name2}_eta']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_eta'], f'gae_{name}_eta')
        
        efficiencydic[f'gae_{name}_nvtx']   = ROOT.TGraphAsymmErrors(nbinnvtx ) 
        efficiencydic[f'gae_{name}_nvtx'].SetName(f'gae_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].SetTitle(f'gae_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].GetXaxis().SetTitle(f'h_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_nvtx'].Divide(th1dic[f'h_{name}_nvtx']  , th1dic[f'h_{name2}_nvtx']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_nvtx'], f'gae_{name}_nvtx')
        
        name2=name
    else:
        th1dic[f'h_{name}_pt']=ROOT.TH1D("pt"  , f'h_{name}_pt;muon p_T [GeV];Events',  20,  20.0, 120.0)
        TH1_fill_and_plot(th1dic[f'h_{name}_pt'], muon.muon_pt[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_pt')
        th1dic[f'h_{name}_eta']=ROOT.TH1D("eta" , f'h_{name}_eta;muon #eta;Events',  24,  -2.4,   2.4)
        TH1_fill_and_plot(th1dic[f'h_{name}_eta'], muon.muon_eta[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_eta')
        th1dic[f'h_{name}_nvtx'] =ROOT.TH1D("nvtx", f'h_{name}_nvtx;N. vertices;Events',  90,   0.5,  90.5)
        TH1_fill_and_plot(th1dic[f'h_{name}_nvtx'], muon.pv_npvsgood[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_nvtx')
        
        nbinpt   = th1dic['h_rec_pt'].GetNbinsX() 
        nbineta  = th1dic['h_rec_eta'].GetNbinsX() 
        nbinnvtx = th1dic['h_rec_nvtx'].GetNbinsX()
        
        efficiencydic[f'gae_{name}_pt']   = ROOT.TGraphAsymmErrors(nbinpt  ) 
        efficiencydic[f'gae_{name}_pt'].SetName(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].SetTitle(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].GetXaxis().SetTitle(f'h_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_pt'].Divide(th1dic[f'h_{name}_pt']  , th1dic[f'h_{name2}_pt']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_pt'], f'gae_{name}_pt')
        
        efficiencydic[f'gae_{name}_eta']   = ROOT.TGraphAsymmErrors(nbineta ) 
        efficiencydic[f'gae_{name}_eta'].SetName(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_eta'].SetTitle(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_eta'].GetXaxis().SetTitle(f'h_{name}_eta')
        efficiencydic[f'gae_{name}_eta'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_eta'].Divide(th1dic[f'h_{name}_eta']  , th1dic[f'h_{name2}_eta']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_eta'], f'gae_{name}_eta')
        
        efficiencydic[f'gae_{name}_nvtx']   = ROOT.TGraphAsymmErrors(nbinnvtx ) 
        efficiencydic[f'gae_{name}_nvtx'].SetName(f'gae_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].SetTitle(f'gae_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].GetXaxis().SetTitle(f'h_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_nvtx'].Divide(th1dic[f'h_{name}_nvtx']  , th1dic[f'h_{name2}_nvtx']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_nvtx'], f'gae_{name}_nvtx')
        
        name2=name


Questions 1:
* In the tag selection, you requested that the tag muon be matched to a trigger object. Why is it important to make sure that the tag has fired a trigger?
* In the MC-truth approach, you can use both muons in the event to measure the efficiency. In the tag-and-probe, you only use the probe. Does this mean that you can use only one muon per event?
* When you factorize the efficiency measurement as explained above, the sample you use to measure each efficiency is a subset of the samples used in the previous steps. Therefore the statistical uncertainties on the various efficiencies are correlated among each other. Does this mean the total uncertainty on the full efficiency becomes larger when you factorize? Why?
* Do you observe significant differences between the "true" efficiencies and the tag-and-probe ones? In what plots and in what regions? What is the maximum size of the differences?
What factors might impact or bias the tag-and-probe efficiency estimate?

# Step 3: Run the "Tag-and-Probe" analyzer on real data

As already mentioned, the tag-and-probe approach is data driven. The analyzer does not access any generator-level information, thus you can safely run it on real data. The following sample--Run2016G_SingleMuon_NANOAOD.root is taken from the 2016G CMS data set, SingleMuon stream, requiring at least a good tag candidate and a good probe candidate per event.

In [ ]:
#file=uproot.open('~/Downloads/Run2016G_SingleMuon_NANOAOD.root')['Events']
file=uproot.open('/cms/store/user/hats/Muon/Run2016G_SingleMuon_NANOAOD.root')['Events']

In [ ]:
muon=ak.Array(ak.zip({"muon_pt":file['Muon_pt'].array(), "muon_eta": file['Muon_eta'].array(), "muon_phi": file['Muon_phi'].array(),
                        "muon_isglobal": file['Muon_isGlobal'].array(), "muon_istracker": file['Muon_isTracker'].array(),
                        "muon_mass": file['Muon_mass'].array(),
                        "muon_charge": file['Muon_charge'].array(),
                        "muon_dxy": file['Muon_dxy'].array(),
                        "muon_dz": file['Muon_dz'].array(),
                        "muon_pfreliso04_all": file['Muon_pfRelIso04_all'].array(), 
                        "muon_tkreliso": file['Muon_tkRelIso'].array(),
                        "muon_looseid": file['Muon_looseId'].array(), 
                        "muon_mediumid": file['Muon_mediumId'].array(), 
                        "muon_tightid": file['Muon_tightId'].array(), 
                        "pv_npvsgood": file['PV_npvsGood'].array(),
                     }))



In [ ]:
trigger_muon=ak.Array(ak.zip({"pt":file['TrigObj_pt'].array()[abs(file['TrigObj_id'].array())==13],
                                "eta": file['TrigObj_eta'].array()[abs(file['TrigObj_id'].array())==13], 
                                "phi": file['TrigObj_phi'].array()[abs(file['TrigObj_id'].array())==13]}))

In [ ]:
trigger_match_mask=deltar_matching(trigger_muon.eta, trigger_muon.phi, muon.muon_eta, muon.muon_phi) #match to a trigger muon

In [ ]:
tag_muon_mask=(muon.muon_pt>25)& (abs(muon.muon_eta)<2.4)& (muon.muon_tightid==1)& (muon.muon_dz<0.1)& (muon.muon_pfreliso04_all< 0.12)& trigger_match_mask

In [ ]:
tag_muon=ak.Array(ak.zip({"pt": muon.muon_pt[tag_muon_mask],
                            "eta": muon.muon_eta[tag_muon_mask],
                            "phi": muon.muon_phi[tag_muon_mask],
                            "mass": muon.muon_mass[tag_muon_mask],
                            "charge": muon.muon_charge[tag_muon_mask]}))

In [ ]:
probe_muon_mask=[]
# make sure the z mass is within 91+-5 window
for tag4s, probe4s, in zip(tag_muon, muon):
    list=[]
    if len(probe4s)==0:
        probe_muon_mask.append([])
        continue
    for probe4 in probe4s:
        bestmass=0
        if len(tag4s)==0:
            list.append(False)
            continue
        for tag4 in tag4s:
            if tag4.charge*probe4.muon_charge>0:
                continue
            mass=sqrt(2*tag4.pt*probe4.muon_pt*(cosh(tag4.eta - probe4.muon_eta) - cos(tag4.phi - probe4.muon_phi)))
            if abs(mass-91.)>5.:
                continue
            else:
                bestmass= mass
                break
        if bestmass!=0:
            list.append(True)
        else:
            list.append(False)
    probe_muon_mask.append(list)

In [ ]:
event_mask_goodpv= file['PV_npvsGood'].array()>0 #at least a good vertex

In [ ]:
event_mask_goodpv_HLT_isomu24= (file['PV_npvsGood'].array()>0)& ((file['HLT_IsoMu24'].array()==1)|(file['HLT_IsoTkMu24'].array()==1))#fired the iso muon trigger

In [ ]:
probe_muon_masking=ak.Array(probe_muon_mask)& (muon.muon_pt>20)& (abs(muon.muon_eta)<2.4)

In [ ]:
muon_mask['rec'].shape

In [ ]:
probe_muon= ak.Array(ak.zip("pt": muon.muon_pt[probe_muon_masking],
                               "eta": muon.muon_eta[probe_muon_masking],
                               "phi": muon.muon_phi[probe_muon_masking],
                               "npvs": muon.pv_npvsgood[probe_muon_masking])

In [ ]:
muon_mask={}
muon_mask['rec']= probe_muon_masking
muon_mask['rec_loose']= probe_muon_masking& (muon.muon_looseid==1) 
muon_mask['rec_medium']= probe_muon_masking& (muon.muon_mediumid==1) 
muon_mask['rec_tight']=  probe_muon_masking& (muon.muon_tightid==1)& (muon.muon_mediumid==1)
muon_mask['rec_tight_iso']= probe_muon_masking& (muon.muon_tightid==1)& (muon.muon_pfreliso04_all< 0.12)& (muon.muon_mediumid==1)
muon_mask['rec_tight_iso_hlt']= probe_muon_masking& (muon.muon_tightid==1)& (muon.muon_pfreliso04_all< 0.12)& trigger_match_mask& (muon.muon_mediumid==1)

In [ ]:
th1dic={}
efficiencydic={}
for name in ['rec', 'rec_loose', 'rec_medium', 'rec_tight', 'rec_tight_iso', 'rec_tight_iso_hlt']:
    if name== 'rec':
        th1dic[f'h_{name}_pt']=ROOT.TH1D("pt"  , f'h_{name}_pt;muon p_T [GeV];Events',  20,  20.0, 120.0)
        TH1_fill_and_plot(th1dic[f'h_{name}_pt'], muon.muon_pt[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_pt')
        th1dic[f'h_{name}_eta']=ROOT.TH1D("eta" , f'h_{name}_eta;muon #eta;Events',  24,  -2.4,   2.4)
        TH1_fill_and_plot(th1dic[f'h_{name}_eta'], muon.muon_eta[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_eta')
        th1dic[f'h_{name}_nvtx'] =ROOT.TH1D("nvtx", f'h_{name}_nvtx;N. vertices;Events',  90,   0.5,  90.5)
        TH1_fill_and_plot(th1dic[f'h_{name}_nvtx'], muon.pv_npvsgood[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_nvtx')
        
        name2=name
        continue
        
    if name== 'rec_tight_iso_hlt':
        th1dic[f'h_{name}_pt']=ROOT.TH1D("pt"  , f'h_{name}_pt;muon p_T [GeV];Events',  20,  20.0, 120.0)
        TH1_fill_and_plot(th1dic[f'h_{name}_pt'], muon.muon_pt[muon_mask[f'{name}']][event_mask_goodpv_HLT_isomu24], f'h_{name}_pt')
        th1dic[f'h_{name}_eta']=ROOT.TH1D("eta" , f'h_{name}_eta;muon #eta;Events',  24,  -2.4,   2.4)
        TH1_fill_and_plot(th1dic[f'h_{name}_eta'], muon.muon_eta[muon_mask[f'{name}']][event_mask_goodpv_HLT_isomu24], f'h_{name}_eta')
        th1dic[f'h_{name}_nvtx'] =ROOT.TH1D("nvtx", f'h_{name}_nvtx;N. vertices;Events',  90,   0.5,  90.5)
        TH1_fill_and_plot(th1dic[f'h_{name}_nvtx'], muon.pv_npvsgood[muon_mask[f'{name}']][event_mask_goodpv_HLT_isomu24], f'h_{name}_nvtx')

        nbinpt   = th1dic['h_rec_pt'].GetNbinsX() 
        nbineta  = th1dic['h_rec_eta'].GetNbinsX() 
        nbinnvtx = th1dic['h_rec_nvtx'].GetNbinsX()
        
        efficiencydic[f'gae_{name}_pt']   = ROOT.TGraphAsymmErrors(nbinpt  ) 
        efficiencydic[f'gae_{name}_pt'].SetName(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].SetTitle(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].GetXaxis().SetTitle(f'h_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_pt'].Divide(th1dic[f'h_{name}_pt']  , th1dic[f'h_{name2}_pt']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_pt'], f'gae_{name}_pt')
        
        efficiencydic[f'gae_{name}_eta']   = ROOT.TGraphAsymmErrors(nbineta ) 
        efficiencydic[f'gae_{name}_eta'].SetName(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_eta'].SetTitle(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_eta'].GetXaxis().SetTitle(f'h_{name}_eta')
        efficiencydic[f'gae_{name}_eta'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_eta'].Divide(th1dic[f'h_{name}_eta']  , th1dic[f'h_{name2}_eta']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_eta'], f'gae_{name}_eta')
        
        efficiencydic[f'gae_{name}_nvtx']   = ROOT.TGraphAsymmErrors(nbinnvtx ) 
        efficiencydic[f'gae_{name}_nvtx'].SetName(f'gae_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].SetTitle(f'gae_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].GetXaxis().SetTitle(f'h_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_nvtx'].Divide(th1dic[f'h_{name}_nvtx']  , th1dic[f'h_{name2}_nvtx']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_nvtx'], f'gae_{name}_nvtx')
        
        name2=name
    else:
        th1dic[f'h_{name}_pt']=ROOT.TH1D("pt"  , f'h_{name}_pt;muon p_T [GeV];Events',  20,  20.0, 120.0)
        TH1_fill_and_plot(th1dic[f'h_{name}_pt'], muon.muon_pt[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_pt')
        th1dic[f'h_{name}_eta']=ROOT.TH1D("eta" , f'h_{name}_eta;muon #eta;Events',  24,  -2.4,   2.4)
        TH1_fill_and_plot(th1dic[f'h_{name}_eta'], muon.muon_eta[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_eta')
        th1dic[f'h_{name}_nvtx'] =ROOT.TH1D("nvtx", f'h_{name}_nvtx;N. vertices;Events',  90,   0.5,  90.5)
        TH1_fill_and_plot(th1dic[f'h_{name}_nvtx'], muon.pv_npvsgood[muon_mask[f'{name}']][event_mask_goodpv], f'h_{name}_nvtx')
        
        nbinpt   = th1dic['h_rec_pt'].GetNbinsX() 
        nbineta  = th1dic['h_rec_eta'].GetNbinsX() 
        nbinnvtx = th1dic['h_rec_nvtx'].GetNbinsX()
        
        efficiencydic[f'gae_{name}_pt']   = ROOT.TGraphAsymmErrors(nbinpt  ) 
        efficiencydic[f'gae_{name}_pt'].SetName(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].SetTitle(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].GetXaxis().SetTitle(f'h_{name}_pt')
        efficiencydic[f'gae_{name}_pt'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_pt'].Divide(th1dic[f'h_{name}_pt']  , th1dic[f'h_{name2}_pt']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_pt'], f'gae_{name}_pt')
        
        efficiencydic[f'gae_{name}_eta']   = ROOT.TGraphAsymmErrors(nbineta ) 
        efficiencydic[f'gae_{name}_eta'].SetName(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_eta'].SetTitle(f'gae_{name}_pt')
        efficiencydic[f'gae_{name}_eta'].GetXaxis().SetTitle(f'h_{name}_eta')
        efficiencydic[f'gae_{name}_eta'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_eta'].Divide(th1dic[f'h_{name}_eta']  , th1dic[f'h_{name2}_eta']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_eta'], f'gae_{name}_eta')
        
        efficiencydic[f'gae_{name}_nvtx']   = ROOT.TGraphAsymmErrors(nbinnvtx ) 
        efficiencydic[f'gae_{name}_nvtx'].SetName(f'gae_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].SetTitle(f'gae_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].GetXaxis().SetTitle(f'h_{name}_nvtx')
        efficiencydic[f'gae_{name}_nvtx'].GetYaxis().SetTitle('Efficiency')
        try:
            efficiencydic[f'gae_{name}_nvtx'].Divide(th1dic[f'h_{name}_nvtx']  , th1dic[f'h_{name2}_nvtx']  , "cl=0.683 b(1,1) mode")
        except:
            print("An exception occurred")
        eff_fill_and_plot(efficiencydic[f'gae_{name}_nvtx'], f'gae_{name}_nvtx')
        
        name2=name



Questions 2:
* Are there very significant differences between the data and MC efficiencies? In what plots and regions?
* Based on the invariant mass distributions for different probe selections, which efficiencies are reliable in your opinion? Which are affected by too much background?
* What other factors may cause differences between data and MC efficiencies? Is it possible to eliminate these differences?